In [1]:
#import libraries

import requests       
import pandas as pd   
import numpy as np  
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'
#send HTTP request
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
#create empty list to store result of each section
movie_name = []
year = []
time = []
rating = []
metascore = []
votes = []
gross = []
description = []
Director = []
Stars = []

In [5]:
movie_data = soup.findAll('div', attrs= {'class': 'lister-item mode-advanced'})
print(movie_data)

[<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt0111161"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt0111161/"> <img alt="The Shawshank Redemption" class="loadlate" data-tconst="tt0111161" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BNDE3ODcxYzMtY2YzZC00NmNlLWJiNDMtZDViZWM2MzIxZDYwXkEyXkFqcGdeQXVyNjAwNDUxODI@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt0111161/">The Shawshank Redemption</a>
<span class="lister-item-year text-muted unbold">(1994)</span>
</h3>
<p class="text-muted">
<span class="certificate">A</span>
<span class="ghost">|</span>
<span class="runtime">142 min</span>
<span class="ghost">|</span>
<span class="genre">
Dra

In [17]:
for store in movie_data:
    #getting name of movie
    name = store.h3.a.text
    movie_name.append(name)
    #np.count_nonzero(movie_name)
    
    #getting release date ,here using text.replace() we can remove the bracket.
    year_of_release = store.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
    year.append(year_of_release)
    
    #getting runtime of movie
    runtime = store.p.find('span', class_ = 'runtime').text.replace(' min', '')
    time.append(runtime)
    
    #getting rating of movie
    rate = store.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
    rating.append(rate)
    
    #getting metascore of movie
    meta  = store.find('span', class_ = 'metascore').text.replace(' ', '') if store.find('span', class_ = 'metascore') else 'NO Score'
    metascore.append(meta)
    
    #gross and votes have same attributes,so here created a common variable and then used indexing to get gross and votes value
    value = store.find_all('span', attrs = {'name': 'nv'})
    
    #getting vote count
    vote = value[0].text
    votes.append(vote)
    
    #getting gross count
    grosses = value[1].text if len(value) >1 else 'No count'
    gross.append(grosses)
    
    #getting description of movie
    desc = store.find_all('p', class_ = 'text-muted')
    description_ = desc[1].text.replace('\n', '') if len(desc) >1 else '*****'
    description.append(description_)
    
    #getting director and stars of movie
    cast = store.find("p", class_ = '')
    cast = cast.text.replace('\n', '').split('|')
    cast = [x.strip() for x in cast]
    cast = [cast[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
    
    Director.append(cast[0])
    Stars.append([x.strip() for x in cast[1].split(",")])
    
    
    

In [13]:
#creating a dataframe
movie_DF = pd.DataFrame({'Name of movie': movie_name, 'Year of relase': year, 'Watchtime': time, 'Movie Rating': rating, 'Metascore': metascore, 'Votes': votes, 'Gross collection': gross, 'Description': description, "Director": Director, 'Star': Stars})


In [19]:
#Save data in excel
movie_DF.to_excel("Top_100_IMDB_Movies.xlsx")
movie_DF.head(20)
 

,Name of movie,Year of relase,Watchtime,Movie Rating,Metascore,Votes,Gross collection,Description,Director,Star
0,The Shawshank Redemption,1994,142,9.3,82,"2,766,234",$28.34M,"Over the course of several years, two convicts...",Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,The Godfather,1972,175,9.2,100,"1,925,242",$134.97M,"Don Vito Corleone, head of a mafia family, dec...",Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,The Dark Knight,2008,152,9.0,84,"2,739,653",$534.86M,When the menace known as the Joker wreaks havo...,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
3,Schindler's List,1993,195,9.0,95,"1,393,356",$96.90M,"In German-occupied Poland during World War II,...",Steven Spielberg,"[Liam Neeson, Ralph Fiennes, Ben Kingsley, Car..."
4,12 Angry Men,1957,96,9.0,97,"820,145",$4.36M,The jury in a New York City murder trial is fr...,Sidney Lumet,"[Henry Fonda, Lee J. Cobb, Martin Balsam, John..."
5,The Lord of the Rings: The Return of the King,2003,201,9.0,94,"1,898,306",$377.85M,Gandalf and Aragorn lead the World of Men agai...,Peter Jackson,"[Elijah Wood, Viggo Mortensen, Ian McKellen, O..."
6,The Godfather Part II,1974,202,9.0,90,"1,309,872",$57.30M,The early life and career of Vito Corleone in ...,Francis Ford Coppola,"[Al Pacino, Robert De Niro, Robert Duvall, Dia..."
7,Spider-Man: Across the Spider-Verse,2023,140,8.9,86,"183,248",#14,"Miles Morales catapults across the Multiverse,...","Directors:Joaquim Dos Santos, Kemp Powers, Jus...","[Shameik Moore, Hailee Steinfeld, Brian Tyree ..."
8,Pulp Fiction,1994,154,8.9,95,"2,123,634",$107.93M,"The lives of two mob hitmen, a boxer, a gangst...",Quentin Tarantino,"[John Travolta, Uma Thurman, Samuel L. Jackson..."
9,Inception,2010,148,8.8,74,"2,431,141",$292.58M,A thief who steals corporate secrets through t...,Christopher Nolan,"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elli..."


In [10]:
np.count_nonzero(movie_name)

100